In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from sshtunnel import SSHTunnelForwarder

## Configurando a senha de acesso ao banco de dados

Criar um arquivo `.env` e nele informar

```
SSH_SERVER=test.labqs.ita.br
SSH_SERVER_PORT=2222
SSH_USER=<usuário informado pela IA-TI>
SSH_PASSWORD=<senha no servidor>
GPES_POSTGRES_SERVER=<banco de dados fornecido pelo DBA>
GPES_POSTGRES_PORT=<porta do banco de dados>
GPES_POSTGRES_DB=<nome do banco de dados>
GPES_POSTGRES_USER=<usuário no banco de dados>
GPES_POSTGRES_PASSWORD=<senha do usuário no banco de dados>
GPES_WEBDAV_USER=<usuário no servidor de arquivos>
GPES_WEBDAV_PASSWORD=<senha do usuário no servidor de arquivos>
GPES_WEBDAV_SERVER=<servidor de arquivos>
GEPS_WEBDAV_PATH=<caminho no servidor de arquivos>
```

### Exemplo

```
SSH_SERVER=test.labqs.ita.br
SSH_SERVER_PORT=2222
SSH_USER=user_temp
SSH_PASSWORD=p4ssw0rd
GPES_POSTGRES_SERVER=postgres01
GPES_POSTGRES_PORT=5432
GPES_POSTGRES_DB=flualfa
GPES_POSTGRES_USER=mecita_ro
GPES_POSTGRES_PASSWORD=p4ssw0rd
GPES_WEBDAV_USER=gpes
GPES_WEBDAV_PASSWORD=s3cr37p4ssw0rd
GPES_WEBDAV_SERVER=test.labqs.ita.br/nextcloud/remote.php/dav/files/gpes
GEPS_WEBDAV_PATH=/flualfa-ita/documents
```

## Obter configurações

In [2]:
import decouple

config = decouple.AutoConfig(' ')

gpes_ssh_server = config('SSH_SERVER')
gpes_ssh_port = int(config('SSH_SERVER_PORT'))
gpes_ssh_user = config('SSH_USER')
gpes_ssh_password = config('SSH_PASSWORD')
db_server = config('GPES_POSTGRES_SERVER')
db_name=config('GPES_POSTGRES_DB')
db_port = int(config('GPES_POSTGRES_PORT'))
db_user = config('GPES_POSTGRES_USER')
db_password = config('GPES_POSTGRES_PASSWORD')
webdav_user = config('GPES_WEBDAV_USER')
webdav_password = config('GPES_WEBDAV_PASSWORD')
webdav_server = config('GPES_WEBDAV_SERVER')
webdav_path = config('GPES_WEBDAV_PATH')

## Abrir túnel com servidor de banco de dados

In [4]:
tunel = SSHTunnelForwarder(
    (gpes_ssh_server, gpes_ssh_port),
    ssh_username = gpes_ssh_user,
    ssh_password = gpes_ssh_password,
    remote_bind_address = (db_server, db_port))

tunel.start()

## Conectar ao servidor

In [5]:
cs = f'postgresql+psycopg2://{db_user}:{db_password}@{tunel.local_bind_host}:{tunel.local_bind_port}/{db_name}'

dbEngine = create_engine(cs, pool_recycle = 3600);
dbConnection = dbEngine.connect();

## Executar a consulta usando Pandas e criando um DataFrame

In [6]:
queryTesteFile = open("teste_Postgres.sql", "r")
queryTeste = queryTesteFile.read()

dfTestTEMP = pd.read_sql(queryTeste, dbConnection);

## Mostrar o resultado da consulta

In [7]:
dfTestTEMP

,id,aht_tat_id,aht_alu_id,aht_tst_id,aht_tur_id,aht_esc_id,aht_cid_id,aht_est_id,aht_users_id_professor,aht_users_id_aplicador,...,aht_ind32,aht_arqaudio1,aht_arqaudio2,aht_arqaudio3,aht_status,aht_statusarq1,aht_statusarq2,aht_statusarq3,created_at,updated_at
0,1,2,1,1,1,79262,73,1,1,2,...,0,public/documents/a1arq1.wav,public/documents/a1arq2.wav,public/documents/a1arq3.wav,2,1,1,1,2022-09-06 14:30:46,2022-09-27 13:13:45
1,2,2,2,1,1,79262,73,1,1,2,...,0,public/documents/a2arq1.wav,public/documents/a2arq2.wav,public/documents/a2arq3.wav,2,1,1,1,2022-09-06 14:30:46,2022-09-27 15:17:20
2,3,2,3,1,1,79262,73,1,1,2,...,0,public/documents/a3arq1.wav,public/documents/a3arq2.wav,public/documents/a3arq3.wav,2,1,1,1,2022-09-06 14:30:46,2022-10-01 11:44:38
3,4,2,4,1,1,79262,73,1,1,2,...,0,public/documents/a4arq1.wav,public/documents/a4arq2.wav,public/documents/a4arq3.wav,2,1,1,1,2022-09-06 14:30:46,2022-10-01 12:25:45
4,5,2,5,1,1,79262,73,1,1,2,...,0,public/documents/a5arq1.wav,public/documents/a5arq2.wav,public/documents/a5arq3.wav,2,1,1,1,2022-09-06 14:30:46,2022-10-03 13:04:02
5,19,5,13,1,12,79722,73,1,4,4,...,0,,,,0,0,0,0,2022-10-14 14:24:38,2022-10-14 14:24:38
6,20,5,14,1,12,79722,73,1,4,4,...,0,,,,0,0,0,0,2022-10-14 14:24:38,2022-10-14 14:24:38
7,11,4,1,1,1,79262,73,1,3,3,...,0,,,,0,0,0,0,2022-10-14 14:22:36,2022-10-14 14:22:36
8,12,4,2,1,1,79262,73,1,3,3,...,0,,,,0,0,0,0,2022-10-14 14:22:36,2022-10-14 14:22:36
9,13,4,3,1,1,79262,73,1,3,3,...,0,,,,0,0,0,0,2022-10-14 14:22:36,2022-10-14 14:22:36


## Fechar conexão e túnel

In [ ]:
dbConnection.close()
dbEngine.dispose()

tunel.close()

## Lista arquivos no repositório e faz download

In [ ]:
from webdav4.client import Client
import os

client = Client("https://" + webdav_server, auth=(webdav_user, webdav_password))

lista_audios = client.ls(webdav_path)

for audio in lista_audios:
    display(f'Downloading {audio["name"]}...')
    client.download_file(audio["name"], "./" + os.path.basename(audio["name"]))

> End